In [ ]:
from tools.sms import Sms
server_url = "https://api.haozhuma.com"
username = "testlinkedin123"
password = "testlinkedin123a"
sid = 72510
sms = Sms(server_url, username, password,sid=72510)

In [ ]:
"""
完整流程:
- 获取手机号码
- 创建浏览器窗口
- 打开页面
- 操作注册 -> 获取注册信息
- 执行注册
- 收信
- 注册成功 -> 写入数据库 -> 设置分组(新注册)
- 注册失败 -> 删除窗口
"""

import random
from faker import Faker
from tools.bit_api import openBrowser,createBrowser,deleteBrowser
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time

sms.release_all_phones()

phone = sms.get_phone()

# 初始化注册信息
faker = Faker('zh_TW')
name = faker.name_female()
firstname = name[1:]
lastname = name[0:1]
year = str(random.randint(1985,2005))
month = str(random.randint(1,12))
day = str(random.randint(1,28))
fb_password = faker.password(length=15, special_chars=False, digits=True, upper_case=True, lower_case=True)

proxy_port = 12233
proxy_server = 'pr.aa4koj1o.lunaproxy.net'
proxy_username = f'user-lu7069006-region-hk-sessid-{fb_password}-sesstime-30'
proxy_password = f'B8#U&E*asjXRg$'

print(phone,name,year,month,day,fb_password)
reg_info = f'{phone},{firstname},{lastname},{year},{month},{day},{fb_password}'

win_name = f'facebook-{phone}'

json_data = {
    'name': win_name,  # 窗口名称
    'remark': '',  # 备注
    # 'proxyMethod': 3,  # 代理方式 2自定义 3 提取IP
    # # 代理类型  ['noproxy', 'http', 'https', 'socks5', 'ssh']
    # 'proxyType': 'socks5',

    # 'dynamicIpUrl': 'http://43.156.34.150:777/proxy/get_proxy_list?key=a850b22464e69475ac69b916dfe1307e&index=9&num=1',
    "proxyMethod": 2,
    "proxyType": "socks5",
    "host": proxy_server,
    "port": proxy_port,
    "proxyUserName": proxy_username,
    "proxyPassword": proxy_password,
    
    'dynamicIpChannel': 'common',
    'isDynamicIpChangeIp': True,
    "browserFingerPrint": {  # 指纹对象
        'coreVersion': '124'  # 内核版本，注意，win7/win8/winserver 2012 已经不支持112及以上内核了，无法打开
    },
    # "platform":"https://www.facebook.com",
    # "userName":phone,
    # "password":fb_password,
    "remark":reg_info
}


browser_id = createBrowser(json_data)

# /browser/open 接口会返回 selenium使用的http地址，以及webdriver的path，直接使用即可
res = openBrowser(browser_id) # 窗口ID从窗口配置界面中复制，或者api创建后返回

# time.sleep(60)

# deleteBrowser(browser_id)

In [ ]:
import random

def connect_selenium(driverPath,debuggerAddress):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("debuggerAddress", debuggerAddress)

    chrome_service = Service(driverPath)
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver



driverPath = res['data']['driver']
debuggerAddress = res['data']['http']
driver = connect_selenium(driverPath,debuggerAddress)
driver.get('https://www.facebook.com/')



In [ ]:

def click_home_reg_btn(driver):
    reg_button = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.LINK_TEXT, "建立新帳戶"))
    )
    time.sleep(random.uniform(3,6))
    reg_button.click()
    time.sleep(random.uniform(3,6))
click_home_reg_btn(driver)

In [ ]:

def input_reg_form(driver,phone,lastname,firstname,password,year,month,day):
    username_input = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "reg_email__"))
    )
    lastname_input = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "lastname"))
    )
    firstname_input = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "firstname"))
    )
    password_input = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "reg_passwd__"))
    )
    year_choose = Select(WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "birthday_year"))
    ))
    month_choose = Select(WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "birthday_month"))
    ))

    day_choose = Select(WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "birthday_day"))
    ))
    sex_click = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.NAME, "sex"))
    )
    time.sleep(random.uniform(2,3))
    username_input.send_keys(phone)
    time.sleep(random.uniform(2,3))
    lastname_input.send_keys(lastname)
    time.sleep(random.uniform(2,3))
    firstname_input.send_keys(firstname)
    time.sleep(random.uniform(2,3))
    password_input.send_keys(password)
    time.sleep(random.uniform(2,3))
    year_choose.select_by_value(year)
    time.sleep(random.uniform(2,3))
    month_choose.select_by_value(month)
    time.sleep(random.uniform(2,3))
    day_choose.select_by_value(day)
    time.sleep(random.uniform(2,3))
    sex_click.click() # 默认注册女性

# print(month)
input_reg_form(driver,phone=f"86{phone}",
               lastname=lastname,
               firstname=firstname,
               password=fb_password,
               year=year,
               month=month,
               day=day)

In [ ]:

# time.sleep(random.uniform(3,4))
submit_reg = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "websubmit"))
)
submit_reg.click()
time.sleep(random.uniform(15,30))

In [ ]:
try:
    reg_tip_span = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "uiHeaderTitle"))
    )
    tip = reg_tip_span.text
    if tip != '輸入短訊中的確認碼':
        print(f'注册失败~{tip},号码拉黑')
    else:
        print(f"注册成功：接收短信验证码")
except Exception as e:
    # res = sms.blacklist_phone(phone)
    # deleteBrowser(browser_id)
    print(f'注册失败~,号码{phone}拉黑,删除窗口')

In [ ]:
phone = '13166430182'
res = sms.get_message(phone)
# {'msg': 'success', 'code': '0', 'sms': '【Facebook】FB-35198 是你的 Facebook 確認碼', 'yzm': '35198'}
if isinstance(res,dict) and res.get('yzm'):
    sms_code = res.get('yzm')
    print(f'{phone}取到验证码:{sms_code}')
print(res)

In [ ]:

#TODO 点击重发/

In [ ]:
sms_code = "64288"
sms_input = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "code"))
)
sms_input.send_keys(sms_code)

In [ ]:
success_reg_btn = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "confirm"))
)
success_reg_btn.click()

In [ ]:
success_reg_btn = WebDriverWait(driver, 90).until(
    EC.presence_of_element_located((By.LINK_TEXT, "確定"))
)
success_reg_btn.click()

In [ ]:
# closecloseBrowser
deleteBrowser(browser_id)